# Word Counts


In [19]:
from pathlib import Path

import pandas as pd

from dolma_count import count_utils

In [5]:
data_dir = Path("../data")
assert data_dir.exists()

In [8]:
!du -h {data_dir}/raw/wiki-en-simple/*

3.8G	../data/raw/wiki-en-simple/en_simple_wiki-0000.json.gz
2.0G	../data/raw/wiki-en-simple/en_simple_wiki-0001.json.gz


In [ ]:
# this is a functional but bad approach to word counting
# I think it's mainly bad because of the sort over approximately 1.5B lines
# not sure of the exact time, but it took a few hours to execute on my mac
# inspired in part by this post: https://stackoverflow.com/questions/10552803/how-to-create-a-frequency-list-of-every-word-in-a-file
!gunzip -c en_simple_wiki-0001.json.gz \
| ggrep -Po '"text":.*?[^\]",' - \
| cut -c 9- \
| sed 's/",$//' \
| sed 's/\n/
/g' \
| sed 's/\"/"/g' \
| tr -d '[[:punct:]]' \
| tr ' ' '
' \
| sort \
| uniq -c \
| sort -nr > wiki0001_word_counts.txt

In [17]:
!echo "a\nb\nc\nd\ne\nb" | python ../src/dolma_count/count_utils.py

a 1
b 2
c 1
d 1
e 1


In [20]:
# this approach counts words at approximately 4M words/second
# at 3.6B tokens in the Dolma wiki data, let's do some back-of-the-envelope calculation
# (3600000000 words / 2) / 4000000 words-per-second / 60s ~= 7.5 minutes
# we would assume that
!gunzip -c {data_dir}/raw/wiki-en-simple/en_simple_wiki-0000.json.gz \
| ggrep -Po '"text":.*?[^\]",' - \
| cut -c 9- \
| sed 's/",$//' \
| sed 's/\n/
/g' \
| sed 's/\"/"/g' \
| tr -d '[[:punct:]]' \
| tr ' ' '
' | python ../src/dolma_count/count_utils.py > {data_dir}/derived/wiki0000_word_counts.txt

Counting words: 1713382367it [07:02, 4058471.98it/s]


In [23]:
!wc -l {data_dir}/derived/wiki0000_word_counts.txt

 9716979 ../data/derived/wiki0000_word_counts.txt


In [34]:
# this command doesn't actually work due the way `sort -k2` works, but it's close enough
!head -n 100000 {data_dir}/derived/wiki0000_word_counts.txt | sort -k2 -nr | head

the 107532442
of 58669363
and 50567064
in 42568367
to 36142078
a 34080892
was 21064791
The 17664703
is 16259858
for 13981699
sort: Broken pipe
